## Advanced Lane Detection

#### Importing the relevant libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob 
% matplotlib inline

#### Defining Classes & Functions

In [2]:
# Define a class to receive the characteristics of each line detection
class Line():
    # initializing function
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]
  
    # Function to find the lane pixels from scratch
    # this uses the historam + sliding window
    def base_find_lane_pixels(self, warped):
        # Take a histogram of the bottom half of the image
        histogram = np.sum(warped[warped.shape[0]//2:,:], axis=0)
        # Create an output image to draw on and visualize the result
        out1_img = np.dstack((warped, warped, warped))
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        if self == leftLaneObject:
            x_base = np.argmax(histogram[:midpoint])
        else:
            x_base = np.argmax(histogram[midpoint:]) + midpoint

        # HYPERPARAMETERS
        # Choose the number of sliding windows
        nwindows = 9
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50

        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(warped.shape[0]//nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated later for each window in nwindows
        x_current = x_base
        
        # Create empty lists to receive left and right lane pixel indices
        lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = warped.shape[0] - (window+1)*window_height
            win_y_high = warped.shape[0] - window*window_height
            ### TO-DO: Find the four below boundaries of the window ###
            win_x_low = x_current - margin
            win_x_high = x_current + margin

            # Identify the nonzero pixels in x and y within the window ###
            good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                (nonzerox >= win_x_low) &  (nonzerox < win_x_high)).nonzero()[0]

            # Append these indices to the lists
            lane_inds.append(good_inds)

            # Recenter next window on their mean position
            if len(good_inds) > minpix:
                    x_current = np.int(np.mean(nonzerox[good_inds]))

        # Concatenate the arrays of indices (previously was a list of lists of pixels)
        try:
            lane_inds = np.concatenate(lane_inds)
        except ValueError:
            # Avoids an error if the above is not implemented fully
            pass

        # Extract left and right line pixel positions
        x = nonzerox[lane_inds]
        y = nonzeroy[lane_inds]
        self.detected = True
        
        return x, y, out1_img
    
    # Function to find the lane pixels from using sliding window
    # around the fitted polynomial from the previous frame
    def previous_find_lane_pixels(self, warped):
        # HYPERPARAMETER
        # Choose the width of the margin around the previous polynomial to search
        # The quiz grader expects 100 here, but feel free to tune on your own!
        margin = 50
        out1_img = np.dstack((warped, warped, warped))

        # Grab activated pixels
        nonzero = warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        fit = self.current_fit
        # Set the area of search based on activated x-values
        # within the +/- margin of our polynomial function
        lane_inds = ((nonzerox > (fit[0]*(nonzeroy**2) + fit[1]*nonzeroy + 
                        fit[2] - margin)) & (nonzerox < (fit[0]*(nonzeroy**2) + 
                        fit[1]*nonzeroy + fit[2] + margin)))

        # extract line pixel positions
        x = nonzerox[lane_inds]
        y = nonzeroy[lane_inds]
        if (np.sum(x) == 0):
            self.detected = False
            x, y, out1_img = self.base_find_lane_pixels(warped)
        else:
            self.detected = True
        
        return x, y, out1_img

    def fit_polynomial(self,warped):
        # Identify if a lane was detected in the previous frame
        # and run the appropriate lane finding algorithm
        if self.detected == False :
            #print("False")
            x, y, out_img = self.base_find_lane_pixels(warped)
        
        elif self.detected == True :
            #print("True")
            x, y, out_img = self.previous_find_lane_pixels(warped)

        # Fit a second order polynomial to each using `np.polyfit` 
        fit = np.polyfit(y, x, 2)
        self.current_fit = fit

        # Generate x and y values for plotting
        ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
        try:
            fitx = fit[0]*ploty**2 + fit[1]*ploty + fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            fitx = 1*ploty**2 + 1*ploty

        return out_img, fit
        
    # calculate the curvature of the lane
    def radius_curvature(self,warped, fit):
        ploty = np.linspace(0,warped.shape[0]-1,warped.shape[0] )
        fitx = fit[0]*ploty**2 + fit[1]*ploty + fit[2]

        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        y_eval = np.max(ploty)

        # Fit new polynomials to x,y in world space
        fit_cr = np.polyfit(ploty*ym_per_pix, fitx*xm_per_pix, 2)

        # Calculate the new radii of curvature
        curvature =  ((1 + (2*fit_cr[0] *y_eval*ym_per_pix + fit_cr[1])**2) **1.5) / np.absolute(2*fit_cr[0])

        # Calculate vehicle center
        lane_bottom = (fit[0]*y_eval)**2 + fit[0]*y_eval + fit[2]

        # Now our radius of curvature is in meters
        return curvature,ploty,fitx,lane_bottom

#### Function Definitions

In [3]:
# Returns the undistorted image given the camera 
# matrix & distortion coefficients
def undistort_image(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)


# Defines a function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.
# RETURNS: binary image with applied threshold
def abs_sobel_thresh(img, orient, sobel_kernel=3, thresh=(0, 255)):  
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

# Defines a function that calculates the magnitude of the 
# x & y gradients and applies the threshold
# RETURNS: binary image with applied magnitude threshold
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    return binary_output

# Defines a function that calculates the direction of the 
# gradient and applies the threshold
# RETURNS: binary image with applied directional threshold
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    return binary_output

# Creates a pipeline which produces an image which is a 
# combination of binary thresholding the S channel (HLS) and 
# binary thresholding the result of applying the Sobel operator 
# in the x direction on the original image.
# RETURNS: binary images with l channel thesholding and s channel
#          thresholding applied
def hls_threshold(img, s_thresh=(170, 245), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel

    return sxbinary, s_binary


# Defines function that will transform given image to a bird's eye view
# RETURNS : transformed image
def transform_image(img): 

    # Grab the image shape
    img_size = (img.shape[1], img.shape[0])

    leftupperpoint  = [570,470]
    rightupperpoint = [720,470]
    leftlowerpoint  = [260,680]
    rightlowerpoint = [1045,680]
    src = np.float32([leftupperpoint, leftlowerpoint, rightupperpoint, rightlowerpoint])
    dst = np.float32([[200,0], [200,680], [1000,0], [1000,680]])

    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)

    return warped, M


### Camera Calibration

In [4]:
images = glob.glob('camera_cal/calibration*.jpg')
nx = 9
ny = 6
objpoints = []
imgpoints = []
objp = np.zeros((9*6,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
img_size=None

for fname in images:
    
    img = mpimg.imread(fname)
    img_size = (img.shape[1], img.shape[0])

    # Convert to grayscale
    undist = np.copy(img)  # Delete this line
    gray = cv2.cvtColor(undist,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    # If found, draw corners
   
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)
        # Draw and display the corners
        cv2.drawChessboardCorners(undist, (nx, ny), corners, ret)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
dst = cv2.undistort(img, mtx, dist, None, mtx)

### Color Spaces and Gradient Thresholding for each Frame

In [5]:
# Gradient and color spaces thresholding
def process_image(img):
    
    undist_img = undistort_image(img, mtx, dist)
    image = np.copy(img)

    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    mag_binary = mag_thresh(image, sobel_kernel=9, mag_thresh=(30, 100))
    dir_binary = dir_threshold(image, sobel_kernel=15, thresh=(0.7, 1.3))
    sxbinary,s_binary = hls_threshold(image)

    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

    # B&W binary
    final_combined_binary = np.zeros_like(dir_binary)
    final_combined_binary[(combined == 1) | (sxbinary == 1) | (s_binary == 1)] = 1

    # color binary
    color_binary = np.dstack((combined, sxbinary, s_binary)) * 255
    warped,M = transform_image(final_combined_binary)

    # identify the fits for the left and right lanes
    out_img,left_fit = leftLaneObject.fit_polynomial(warped)
    out_img,right_fit = rightLaneObject.fit_polynomial(warped)
    
    # calculate the radius of curvature
    left_curvature, ploty, left_fitx, left_lane_bottom = leftLaneObject.radius_curvature(warped, left_fit)
    right_curvature, ploty, right_fitx, right_lane_bottom = rightLaneObject.radius_curvature(warped, right_fit)
    
    # Lane center as mid of left and right lane bottom 
    # Average cuvature of the fits
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    lane_center = (left_lane_bottom + right_lane_bottom)/2.
    avg_curvature = (left_curvature + right_curvature)/2.
    center_image = 640
    center = (lane_center - center_image)*xm_per_pix #Convert to meters
    position = "left" if center < 0 else "right"
    center = "Vehicle is {:.2f}m {} of center".format(center, position)
    curve = "Radius of Curvature = {:.2f}(m)".format(avg_curvature)
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    leftupperpoint  = [570,470]
    rightupperpoint = [720,470]
    leftlowerpoint  = [260,680]
    rightlowerpoint = [1045,680]
    src = np.float32([leftupperpoint, leftlowerpoint, rightupperpoint, rightlowerpoint])
    dst = np.float32([[200,0], [200,680], [1000,0], [1000,680]])

    Minv = cv2.getPerspectiveTransform(dst, src)

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    
    result = cv2.addWeighted(undist_img, 1, newwarp, 0.3, 0)
    #Adding text to the image for each frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, center, (230, 50), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(result, curve, (230, 100), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return result

In [7]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Initializing line objects for left and right lanes
leftLaneObject = Line()
rightLaneObject = Line()

output = 'output_video/project_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("./project_video.mp4").subclip(0,5)
clip1 = VideoFileClip("./project_video.mp4")
clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video output_video/project_video_out.mp4
[MoviePy] Writing video output_video/project_video_out.mp4


100%|█████████▉| 1260/1261 [07:33<00:00,  1.65it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video/project_video_out.mp4 

CPU times: user 15min 21s, sys: 54.8 s, total: 16min 16s
Wall time: 7min 34s
